# Interactive Tidal Data Point Query

Click on the map to query detailed tidal data at specific coordinates.

This notebook:
1. Shows grid coverage boundaries (aggregated for performance)
2. Captures clicked coordinates automatically
3. Queries the nearest data point and displays summary information

In [ ]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import folium
from IPython.display import display

from query_tidal_manifest import TidalManifestQuery

## Configuration

In [ ]:
# Configuration
MANIFEST_PATH = Path(
    "/projects/hindcastra/Tidal/datasets/high_resolution_tidal_hindcast/manifests/v0.3.0/manifest.json"
)
BASE_DATA_DIR = Path("/projects/hindcastra/Tidal/datasets/high_resolution_tidal_hindcast")

# For local development, override paths if needed:
# MANIFEST_PATH = Path("./manifests/v0.3.0/manifest.json")
# BASE_DATA_DIR = Path("./data")

<cell_type>markdown</cell_type>## Step 1: Load Manifest and Generate Location Boundaries

The manifest contains per-location geospatial bounds derived from the FVCOM mesh geometry.

In [ ]:
# Load manifest and generate location boundaries
print(f"Loading manifest from: {MANIFEST_PATH}")
query = TidalManifestQuery(MANIFEST_PATH)

print(f"\nGenerating location boundaries from manifest...")
geojson = query.generate_location_boundaries_geojson()

## Step 2: Display Interactive Map

**Instructions:** Click anywhere on the map to select a point for querying.

In [ ]:
# Create map centered on data
bounds = query.manifest['spatial_bounds']
center_lat = (bounds['lat_min'] + bounds['lat_max']) / 2
center_lon = (bounds['lon_min'] + bounds['lon_max']) / 2

# Initialize map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=5,
    tiles='OpenStreetMap'
)

# Add location boundaries as GeoJSON layer with tooltips
folium.GeoJson(
    geojson,
    name='Location Boundaries',
    style_function=lambda x: {
        'fillColor': '#3388ff',
        'color': '#3388ff',
        'weight': 2,
        'fillOpacity': 0.2
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['label', 'location', 'point_count'],
        aliases=['Location:', 'ID:', 'Points:'],
        localize=True
    )
).add_to(m)

# Add custom click handler with copyable code snippet
click_js = """
<script>
function onMapClick(e) {
    var lat = e.latlng.lat.toFixed(6);
    var lon = e.latlng.lng.toFixed(6);
    
    var codeSnippet = 'QUERY_LAT = ' + lat + '\\nQUERY_LON = ' + lon;
    
    var popupContent = `
        <div style="min-width: 300px;">
            <b>📍 Selected Coordinates</b><br><br>
            <div style="background: #f5f5f5; padding: 10px; border-radius: 4px; font-family: monospace; margin: 10px 0;">
                <div>QUERY_LAT = ${lat}</div>
                <div>QUERY_LON = ${lon}</div>
            </div>
            <button onclick="navigator.clipboard.writeText('QUERY_LAT = ${lat}\\nQUERY_LON = ${lon}').then(() => {
                this.innerHTML = '✓ Copied!';
                this.style.background = '#4CAF50';
                setTimeout(() => {
                    this.innerHTML = '📋 Copy to Clipboard';
                    this.style.background = '#2196F3';
                }, 2000);
            })" 
            style="width: 100%; padding: 8px; background: #2196F3; color: white; border: none; border-radius: 4px; cursor: pointer; font-size: 14px;">
                📋 Copy to Clipboard
            </button>
            <div style="margin-top: 10px; font-size: 12px; color: #666;">
                <b>Next steps:</b><br>
                1. Click "Copy to Clipboard"<br>
                2. Scroll to "Step 3" below<br>
                3. Paste into the query cell<br>
                4. Run the cell to see results
            </div>
        </div>
    `;
    
    L.popup()
        .setLatLng(e.latlng)
        .setContent(popupContent)
        .openOn(this);
}
</script>
"""

# Add the JavaScript to the map
m.get_root().html.add_child(folium.Element(click_js))

# Bind click event
click_handler = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    // Wait for map to be ready
    setTimeout(function() {
        var maps = document.querySelectorAll('.folium-map');
        if (maps.length > 0) {
            var mapId = maps[0].id;
            var map = window[mapId];
            if (map) {
                map.on('click', onMapClick);
            }
        }
    }, 1000);
});
</script>
"""
m.get_root().html.add_child(folium.Element(click_handler))

# Add instructions overlay
instructions_html = '''
<div style="position: fixed; 
    top: 10px; left: 50px; width: 350px; 
    background-color: white; border:2px solid #3388ff; 
    z-index:9999; font-size:14px; padding: 10px; border-radius: 5px;">
    <b>📍 Interactive Tidal Data Query</b><br><br>
    <b>Instructions:</b><br>
    1. Hover over location boundaries to see details<br>
    2. Click anywhere on the map to select a point<br>
    3. Click "Copy to Clipboard" in the popup<br>
    4. Scroll to "Step 3" and paste coordinates<br>
    5. Run the query cell to see results
</div>
'''
m.get_root().html.add_child(folium.Element(instructions_html))

print("Map ready! Click on the map to select coordinates.")
m

## Step 3: Query Point Data

Copy the coordinates from the map click and paste them below, then run this cell.

In [ ]:
# ==============================================================================
# PASTE CLICKED COORDINATES HERE
# ==============================================================================
QUERY_LAT = 49.94
QUERY_LON = -174.96

# ==============================================================================
# Query and Display Results
# ==============================================================================
print(f"Querying point: ({QUERY_LAT:.6f}, {QUERY_LON:.6f})")
print("=" * 80)

# Find nearest grid
result = query.query_nearest_point(lat=QUERY_LAT, lon=QUERY_LON, load_details=True)

if result is None:
    print("❌ No data found near this location")
else:
    # Display grid information
    print(f"\n✓ Nearest Grid: {result['grid_id']}")
    print(f"  Centroid: ({result['centroid'][0]:.6f}, {result['centroid'][1]:.6f})")
    print(f"  Distance: {result['distance_deg']:.6f}° (~{result['distance_deg'] * 111:.2f} km)")
    
    details = result['details']
    print(f"\n📍 Location: {details['location']}")
    print(f"⏱️  Temporal Resolution: {details['temporal']}")
    print(f"📊 Points in Grid: {len(details['points'])}")
    
    # Find closest point within grid
    min_distance = float('inf')
    closest_point = None
    
    for point in details['points']:
        lat_diff = point['lat'] - QUERY_LAT
        lon_diff = point['lon'] - QUERY_LON
        distance = np.sqrt(lat_diff**2 + lon_diff**2)
        
        if distance < min_distance:
            min_distance = distance
            closest_point = point
    
    print(f"\n🎯 Closest Data Point:")
    print(f"  Face ID: {closest_point['face']}")
    print(f"  Coordinates: ({closest_point['lat']:.6f}, {closest_point['lon']:.6f})")
    print(f"  Distance: {min_distance:.6f}° (~{min_distance * 111:.2f} km)")
    
    # Construct file path
    location = details['location']
    file_path_str = closest_point['file_path']
    
    if file_path_str.startswith(location + '/'):
        relative_path = file_path_str[len(location) + 1:]
    else:
        relative_path = file_path_str
    
    parquet_file = BASE_DATA_DIR / location / "b4_vap_partition" / relative_path
    
    print(f"\n📁 Parquet File:")
    print(f"  Name: {parquet_file.name}")
    print(f"  Location: {parquet_file.parent}")
    
    if not parquet_file.exists():
        print(f"\n❌ ERROR: File not found at {parquet_file}")
    else:
        # Load data
        print(f"\n📖 Loading data...")
        df = pd.read_parquet(parquet_file)
        
        # Basic info
        print(f"\n📊 Dataset Information:")
        print(f"  Rows: {len(df):,}")
        print(f"  Columns: {len(df.columns)}")
        print(f"  Time range: {df.index.min()} to {df.index.max()}")
        print(f"  File size: {parquet_file.stat().st_size / (1024**2):.2f} MB")
        
        # Display column names
        print(f"\n📋 Columns:")
        for col in df.columns:
            print(f"  - {col}")
        
        # Show head
        print(f"\n🔍 Data Preview (first 5 rows):")
        display(df.head(5))
        
        # Show summary statistics
        print(f"\n📈 Summary Statistics:")
        summary = pd.DataFrame({
            'min': df.min(),
            'mean': df.mean(),
            'max': df.max(),
            'std': df.std(),
            'nan_count': df.isna().sum()
        })
        display(summary)

## Plot Time Series

Visualize the data loaded from the query above:

## Export Data

Save the queried data to a file: